## Library

In [1]:
import simpy
import numpy.random as random
import pandas as pd
import scipy.stats as st
from collections import namedtuple
from copy import deepcopy

## Floor

In [2]:
def id_generator():
    """Generate unique, global customer ID."""
    i = 1
    while True:
        yield i
        i += 1
        
class Customer:
    def __init__(self):
        self.cid = next(id_gen)
        
    def log(self):
        global customers_log
        customers_log.append(vars(self))
        
class IATDistribution:
    def __init__(self, path):
        self.df = pd.read_csv(path)
        self.df["Floor"] = self.df["Floor"].apply(lambda x:x.lstrip('0'))
        
    def getter(self, location, section, direction, floor):
        
        # filter row by mulitple conditions         
        temp = self.df[(self.df['Location'] == location) & (self.df['Section'] == section) & \
                       (self.df['Direction'] == direction) & (self.df['Floor'] == floor)][['Distribution','Parameters']]
        
        # return distribution name and parsed parameters
        return {
            'dist': getattr(st, temp['Distribution'].values[0]),
            'params': self.params_parser(temp['Parameters'].values[0])
        }
    
    @staticmethod
    def params_parser(params_string):
        params = []
        for param in params_string.replace("(", "").replace(")", "").split(", "): 
            params.append(float(param))
        return params

class Queue:
    def __init__(self, env, floor, direction):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.queue_array = []
        self.arrival_event = self.env.event()
        
        # start process
        self.inflow_proc = self.env.process(self.inflow())
        self.outflow_proc = self.env.process(self.outflow())
        
    def inflow(self):
        while True:
            customers = yield self.arrival_event
            print('[INFLOW] Outer Call {} Floor {} '.format(self.floor, 'up' if self.direction == 1 else 'down'))
            
            # disable call if already assigned
            if(len(self.queue_array) == 0):
                global CALL_EVENT
                mission = Mission(direction=self.direction, destination=self.floor)
                CALL_EVENT.succeed(value = mission)
                
                # reactivate event
                CALL_EVENT = self.env.event()
                
            # add customers to waiting queue
            self.queue_array = self.queue_array + customers  
            print('[INFLOW] {} people waiting in {} floor'.format(len(self.queue_array), self.floor))
                          
    def outflow(self):
        while True :
            
            # elevator arrives
            availible, elevIndex = yield AAA_EVENT[self.direction][self.floor]

            riders = []
            if availible == 0:
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11')
                
            while (availible > 0) and (len(self.queue_array) > 0):
                
                customer = self.queue_array.pop()
                customer.boarding_time = self.env.now[0]
                
                riders.append(customer)
                
                availible -= 1
                
            # time the customers take to on board
            yield self.env.timeout(len(riders) * random.randint(CONFIG['WALKING_MIN'], CONFIG['WALKING_MAX']))
            
            print('[OUTFLOW] {} People Enters'.format(len(riders)))
                
            # customers on board
            BBB_EVENT[elevIndex].succeed(value=riders)
            BBB_EVENT[elevIndex] = self.env.event()
class Floor:
    def __init__(self, env, floor, direction, IAT, DD):
        self.env = env
        self.floor = floor
        self.direction = direction
        
        # statistical data
        self.IAT = IAT
        self.DD = DD if len(DD) == 1 else DD/DD.sum()
        
        # start process
        self.queue = Queue(env, self.floor, self.direction)
        self.source_proc = env.process(self.Source(env))
        
    def Source(self, env):
        while True:
            
            # 1. set inter-arrival time based on given distribution
            t = -1
            while t < 0:
                t = self.IAT['dist'].rvs(*self.IAT['params'][:-2],loc = self.IAT['params'][-2],scale = self.IAT['params'][-1], size = 1)
            yield self.env.timeout(t)
            
            # 2. set number of people of arrival group
            customers = []
            for i in range(random.randint(CONFIG['ARRIVAL_MIN'], CONFIG['ARRIVAL_MAX'])):
                
                # 3. set customer destination based on given posibility 
                customer = Customer()
                customer.source = self.floor
                customer.destination = random.choice(self.DD.index, p=self.DD)
                customer.start_time = self.env.now[0]
#                 cid = next(id_gen)

                customers.append(customer)

            self.queue.arrival_event.succeed(value = customers)
            self.queue.arrival_event = self.env.event()


## Elevator

In [103]:
def displacement(floor1, floor2):
    floor1 = int(floor1) if not 'B' in floor1 else -int(floor1[1:]) + 1 
    floor2 = int(floor2) if not 'B' in floor2 else -int(floor2[1:]) + 1
    return floor2-floor1

class AssignmentError(Exception):
    """Exception : Fail to assign a elevator."""
    pass

class IndexError(Exception):
    """Exception : Wrong Index."""
    pass

    
Mission = namedtuple("Mission", ["direction", "destination"])
    
class StopList:
    IDLE = 0; ACTIVE = 1; NA = -1
    
    def __init__(self, FloorList):
        self.floorList = deepcopy(FloorList)
        
        self._list = {
             1: [0] * (len(FloorList)),
            -1: [0] * (len(FloorList))
        }
        
        # dictionary for indexing
        self.index = {
             1:{ floor:index for index, floor in enumerate(         FloorList ) },
            -1:{ floor:index for index, floor in enumerate(reversed(FloorList)) }
        }
        self.reversed_index = {
             1:{ index:floor for index, floor in enumerate(FloorList) },
            -1:{ index:floor for index, floor in enumerate(reversed(FloorList)) }
        }
        print(self._list)
        print(self.index)
        print(self.reversed_index)
    def __str__(self):
        string = 'Stop List: \n  up   down \n'
        sss = ['{} [{}] [{}] {}\n'.format(f, i, j, f) for f, i, j in zip(self.floorList, self._list[1], self._list[-1])]
        string.join(sss)
        return str(self._list)
#     up  down
# 4   [0]  [0]
# 3   [1]  [1]
# 2   [0]  [0]
# 1   [1]  [0]
    
    def isEmpyty(self):
        for i, j in zip(self._list[1], self._list[-1]):
            if i == StopList.ACTIVE or j == StopList.ACTIVE:
                return False
        return True
    
    def pushOuter(self, elev, direction, floor):
        currentIndex = self.index[direction][floor]
        
        # illegal index
        if self._list[direction][currentIndex] == StopList.NA:
            raise IndexError()
        
        # add outer call to stoplist
        self._list[direction][currentIndex] = StopList.ACTIVE
        
    def pushInner(self, elev, destination):
        currentIndex = self.index[elev.direction][destination]
        
        # illegal index
        if self._list[elev.direction][currentIndex] == StopList.NA:
            raise IndexError()
            
        # add inner call to stoplist
        self._list[elev.direction][currentIndex] = StopList.ACTIVE
    
    def pop(self, elev):
        currentIndex = self.index[elev.direction][elev.current_floor]
        
        # illegal index
        if self._list[elev.direction][currentIndex] == StopList.NA:
            raise IndexError()
        
        # remove target floor from stoplist
        self._list[elev.direction][currentIndex] = StopList.IDLE
    
    def next_target(self, elev):
        current_floor = elev.current_floor
        if current_floor == 'B2':
            current_floor = Elevator.forwards(current_floor, elev.direction)

        v = 2
        if v == 3:
            # same direction, front

            curr_index = self.index[elev.direction][current_floor]
            for i, state in enumerate(self._list[elev.direction][curr_index:]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i+curr_index]
                
            # different direction
            for i, state in enumerate(self._list[-elev.direction]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[-elev.direction][i]

            # same direction, back
            for i, state in enumerate(self._list[elev.direction][:curr_index]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i]
            
        if v == 2:
            peak = None   
        
            # same direction, front
            curr_index = self.index[elev.direction][current_floor]
            for i, state in enumerate(self._list[elev.direction][curr_index:]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i+curr_index]

                # compute ligetimate peak
                if state != StopList.NA:
                    peak = self.reversed_index[elev.direction][i+curr_index]

            # different direction
            for i, state in enumerate(self._list[-elev.direction]):
                if state == StopList.ACTIVE:
                    return peak

            # same direction, back
            for i, state in enumerate(self._list[elev.direction][:curr_index]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i]
        
        return None

class Elevator:
    def __init__(self, env, elevIndex, failAssignment, floorList):
        self.env = env
        self.elevIndex = elevIndex
        self.capacity = CONFIG['ELEV_CAPACITY']
        self.riders = []
        self.failAssignment = failAssignment
        
        # schedule list
        self.stop_list = StopList(floorList)
        
        # initial states
        self.current_floor = '1'
        self.direction = 0
        self.assign_event = self.env.event()
        self.finish_event = self.env.event()
        
        # start process
        self.env.process(self.idle())
            
    def idle(self):
        print('[IDLE] Elevator {} Activated'.format(self.elevIndex))
        
        while True:
            
            # first assignment
            mission = yield self.assign_event
            
            # reactivate
            self.assign_event = self.env.event()
            self.finish_event.succeed()
            self.finish_event = self.env.event()
            
            
            print('[IDLE] Outer Call: {}'.format(mission))
            
            # push outer call
            self.stop_list.pushOuter(self, mission.direction, mission.destination)

            # determine initial direction
            if displacement(self.current_floor, mission.destination) > 0:
                self.direction = 1
            elif displacement(self.current_floor, mission.destination) < 0:
                self.direction = -1
            else:
                # set the direction of elevator to that of mission
                self.direction = mission.direction    
            
            
            
            # start onMission process
            yield self.env.process(self.onMission())
            
            print('[IDLE] Elevator {} Stopped.'.format(self.elevIndex))
            
            # return to IDLE state
            self.direction = 0
#             self.resubmit()
    
    def onMission(self):
        while not self.stop_list.isEmpyty():
            print(self.stop_list)
            
            nextTarget = self.stop_list.next_target(self)
            print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
            
            moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            while self.current_floor != nextTarget:
                value = yield self.assign_event | moving_proc
                if self.assign_event.triggered:
                    mission = value[self.assign_event]
                    self.assign_event = self.env.event()
                    
                    self.finish_event.succeed()
                    self.finish_event = self.env.event()
                    
                    print('[ONMISSION] New Assigenment {}'.format(mission))
                    
                    before = nextTarget
                    self.stop_list.pushOuter(self, mission.direction, mission.destination)
                    
                    nextTarget = self.stop_list.next_target(self)
                    print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
                    if before != nextTarget:
                        
                        moving_proc.interrupt()
                        moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            print('[ONMISSION] Arrive At {} Floor'.format(self.current_floor))
            
            yield self.env.process(self.serving())
            print('[Afer Serving]',self.riders)
        
    def moving(self, destination, source):
        """source is needed to account for the acceleration and deceleration rate of the elevator"""
        print('[MOVING] Moving Process Started: to {}'.format(destination))
        try:   
            while self.current_floor != destination:
                
                # determine traveling time for 1 floor
                t = self.travelingTime(destination, self.current_floor, source)
                yield self.env.timeout(t)
                
                # advance 1 floor
                self.current_floor = self.forwards(self.current_floor, self.direction)
                
                print('[MOVING] Update To {} Floor'.format(self.current_floor))
        except simpy.Interrupt as i:
            print('[MOVING] Interrupted')
            print(self.stop_list)
            
        
    def serving(self):
        
        # remove from schedule
        self.stop_list.pop(self)
        print('pop:',self.stop_list)
        
        # customers leave
        leaveCount = 0
        for i in range(len(self.riders)-1, -1, -1):
            if self.riders[i].destination == self.current_floor:
                
                customer = self.riders.pop(i)
                customer.leave_time = self.env.now[0]
                customer.log()
                
                leaveCount += 1
        yield self.env.timeout(leaveCount*1)
        print('[SERVING] {} Customers Leave'.format(leaveCount))
        
        # exclude 'peak' condition
        if not((self.current_floor == FloorList[-1] and self.direction == 1) or \
               (self.current_floor == FloorList[0] and self.direction == -1)):
            
            # elevator arrive
            AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity-len(self.riders), self.elevIndex))
            AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

            # customers on board
            riders = yield BBB_EVENT[self.elevIndex]

            print('[SERVING] Customers Aboard: \n  ', riders)

            # add inner calls
            for customer in riders:
                self.stop_list.pushInner(self, customer.destination)
            self.riders = self.riders + riders
            
            print(self.stop_list)
            
            
            
            global elevator_log
            elevator_log.append({'floor':self.current_floor, 'direction': self.direction, 'time':self.env.now[0]})
        
        # determine direction
        nextTarget = self.stop_list.next_target(self)
        print('[SERVING] NEXT TARGET {}'.format(nextTarget))
        if nextTarget is None:
            return
        else:
            if displacement(self.current_floor, nextTarget) > 0:
                print('change direction: up')
                self.direction = 1
            elif displacement(self.current_floor, nextTarget) < 0:
                print('change direction: down')
                self.direction = -1
            else:
                print('change {} to {}'.format(self.direction, -1*self.direction))
                # make a turn
                self.direction = -1*self.direction
                self.stop_list.pop(self)
                print('p2p:',self.stop_list)             
                
                # customers on board
                self.boarding()

                elevator_log.append({'floor':self.current_floor, 'direction': self.direction, 'time':self.env.now[0]})
        
    def resubmit(self):
        for i in range(len(self.failAssignment)):
            mission = self.failAssignment.pop()
            global RESUBMIT_EVENT
            RESUBMIT_EVENT.succeed(value=mission)
            RESUBMIT_EVENT = self.env.event()
    
    def travelingTime(self, destination, current, source):
        # acceleration should be considered
        return 10
    
    def boarding(self):
        # boarding
        AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity-len(self.riders), self.elevIndex))
        AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

        riders = yield BBB_EVENT[self.elevIndex]

        print('[SERVING] Customers Aboard: \n  ', riders)

        # new customers
        for customer in riders:
            self.stop_list.pushInner(self, customer.destination)
        print(self.stop_list)
        self.riders = self.riders + riders
        
    @staticmethod
    def mission_priority(mission):
        floor = mission.destination
        direction = mission.direction
        index = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        index = index*direction
        return index  
    
    @staticmethod
    def forwards(floor, direction):
        floor = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        floor += direction
        floor = str(floor) if floor > 0 else "B{}".format(abs(floor-1))
        return floor
        
class ElevatorController:
    def __init__(self, env, elevatorList, floorList):
        self.env = env
        self.elevatorList = elevatorList
        self.failAssignment = []
        self.elevators = dict()
        for elevatorName in elevatorList:
            self.elevators[elevatorName] = Elevator(env, elevatorName, self.failAssignment, floorList)
        self.env.process(self.assignCalls())
    
    def assignCalls(self):
        
        while True:
            mission = yield CALL_EVENT# | RESUBMIT_EVENT
            
            try:
#                 candidate = self.bestCandidate(mission.direction, mission.destination)
                candidate = random.choice(self.elevatorList, p= [1/len(self.elevatorList)]*len(self.elevatorList))
        
                self.elevators[candidate].assign_event.succeed(value = mission)
#                 self.elevators[candidate].assign_event = self.env.event()
                
                yield  self.elevators[candidate].finish_event
                
                print('[AssignCalls] assign')
            except AssignmentError:
                print('Fail to Assign')
                
    def bestCandidate(self, direction, source):
        """Assignment Policy"""
        minDistance = 50
        bestElevator = str()
        for elevator in self.elevators.values():
            if (minDistance > abs(displacement(source, elevator.current_floor)) and \
               ((direction == elevator.direction and elevator.current_floor*direction < direction*source) \
                or elevator.direction == 0)):
                    minDistance = abs(displacement(source, elevator.current_floor))
                    bestElevator = elevator.elevIndex
            
        if minDistance == 50:
            self.failAssignment.append(Mission(direction=direction, destination=source))
            raise AssignmentError()
            
        return bestElevator


In [107]:
customers_log = []
elevator_log = []

# Inter-Arrival Time Distribution
IATD = IATDistribution('./BestFitDistribution - 複製.csv')

# Destination Distribution
DD = pd.read_csv('./FloorRatio_NHB.csv').iloc[:,1:].set_index('from').iloc[0:19,0:19]
DD = DD.drop(['B2'], axis=1).drop(['B2'], axis=0)
DD

# Location & Time Section
Location = "北棟病床"; TimeSection = 2

# Floor List
FloorList = ['B4','B3','B1','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']

# Elevator ID
ElevatorList = ['3001']

# Enviornment Variable
env = simpy.Environment()
id_gen = id_generator()
CONFIG = {
    'ARRIVAL_MAX': 2,
    'ARRIVAL_MIN': 1,    
    'WALKING_MAX': 3,
    'WALKING_MIN': 1,
    'RANDOM_SEED': 50,
    'ELEV_CAPACITY': 20
    
}
random.seed(CONFIG['RANDOM_SEED'])

# Global Event
CALL_EVENT = env.event()
RESUBMIT_EVENT = env.event()
AAA_EVENT = { 1: { i : env.event() for i in FloorList}, -1: { i : env.event() for i in FloorList} }
BBB_EVENT = { i: env.event() for i in ElevatorList }

# process
floors_upward   = [ Floor(env, i,  1, IATD.getter(Location, TimeSection,   'up', i), DD.loc[i][i.lstrip("0"):]) for i in FloorList[ :-1]]
floors_downward = [ Floor(env, i, -1, IATD.getter(Location, TimeSection, 'down', i), DD.loc[i][:i.lstrip("0")]) for i in FloorList[1:  ]]
elevator_ctrl = ElevatorController(env, ElevatorList, FloorList)

env.run(until=5000)

{1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{1: {'B3': 0, 'B1': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '10': 11}, -1: {'10': 0, '9': 1, '8': 2, '7': 3, '6': 4, '5': 5, '4': 6, '3': 7, '2': 8, '1': 9, 'B1': 10, 'B3': 11}}
{1: {0: 'B3', 1: 'B1', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9', 11: '10'}, -1: {0: '10', 1: '9', 2: '8', 3: '7', 4: '6', 5: '5', 6: '4', 7: '3', 8: '2', 9: '1', 10: 'B1', 11: 'B3'}}
[IDLE] Elevator 3001 Activated
[INFLOW] Outer Call 5 Floor down 
[INFLOW] 1 people waiting in 5 floor
[IDLE] Outer Call: Mission(direction=-1, destination='5')
{1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -1: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]}
[ONMISSION] NEXT TARGET 10
[MOVING] Moving Process Started: to 10
[AssignCalls] assign
[MOVING] Update To 2 Floor
[MOVING] Update To 3 Floor
[MOVING] Update To 4 Floor
[INFLOW] Outer Call 3 Floor down 
[INFLOW] 1 people waiting in 3 floor
[ONMI

{1: [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], -1: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]}
[SERVING] NEXT TARGET 7
change direction: up
[Afer Serving] [<__main__.Customer object at 0x0000024F82A5C668>, <__main__.Customer object at 0x0000024F82A5CB38>, <__main__.Customer object at 0x0000024F82A5CE48>, <__main__.Customer object at 0x0000024F82D43550>, <__main__.Customer object at 0x0000024F82AC8828>, <__main__.Customer object at 0x0000024F82DA6E10>, <__main__.Customer object at 0x0000024F82D435F8>, <__main__.Customer object at 0x0000024F82A5CD68>, <__main__.Customer object at 0x0000024F82DA6630>]
{1: [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], -1: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]}
[ONMISSION] NEXT TARGET 7
[MOVING] Moving Process Started: to 7
[INFLOW] Outer Call 4 Floor up 
[INFLOW] 1 people waiting in 4 floor
[ONMISSION] New Assigenment Mission(direction=1, destination='4')
[ONMISSION] NEXT TARGET 7
[AssignCalls] assign
[MOVING] Update To 7 Floor
[ONMISSION] Arrive At 7 Floor
pop: {1: [0, 1, 0,

[ONMISSION] New Assigenment Mission(direction=1, destination='1')
[ONMISSION] NEXT TARGET 6
[AssignCalls] assign
[MOVING] Update To 6 Floor
[ONMISSION] Arrive At 6 Floor
pop: {1: [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1], -1: [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]}
[SERVING] 4 Customers Leave
[OUTFLOW] 0 People Enters
[SERVING] Customers Aboard: 
   []
{1: [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1], -1: [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]}
[SERVING] NEXT TARGET 7
change direction: up
[Afer Serving] [<__main__.Customer object at 0x0000024F82D76550>, <__main__.Customer object at 0x0000024F81590BA8>, <__main__.Customer object at 0x0000024F829B1438>, <__main__.Customer object at 0x0000024F82D765F8>, <__main__.Customer object at 0x0000024F829B1BA8>, <__main__.Customer object at 0x0000024F82A5C550>]
{1: [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1], -1: [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]}
[ONMISSION] NEXT TARGET 7
[MOVING] Moving Process Started: to 7
[MOVING] Update To 7 Floor
[ONMISSION] Arrive At 7 Floor
po

[ONMISSION] Arrive At 4 Floor
pop: {1: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], -1: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]}
[SERVING] 1 Customers Leave
[OUTFLOW] 0 People Enters
[SERVING] Customers Aboard: 
   []
{1: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], -1: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]}
[SERVING] NEXT TARGET 3
change direction: down
[Afer Serving] [<__main__.Customer object at 0x0000024F829B1048>, <__main__.Customer object at 0x0000024F82A5C908>]
{1: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], -1: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]}
[ONMISSION] NEXT TARGET 3
[MOVING] Moving Process Started: to 3
[MOVING] Update To 3 Floor
[ONMISSION] Arrive At 3 Floor
pop: {1: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], -1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]}
[SERVING] 0 Customers Leave
[OUTFLOW] 1 People Enters
[SERVING] Customers Aboard: 
{1: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], -1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]}
[SERVING] NEXT TARGET 1
change direction: down
[Afer Serving] [<__main__.Customer ob

In [ ]:
# for customers in [i.queue.queue_array for i in floors_upward]:
#     for customer in customers:
#         print(customer.cid)
# for customers in [i.queue.queue_array for i in floors_downward]:
#     print(len(customers))
#     for customer in customers:
#         print(customer.cid)

# elevator_ctrl.elevators['3001'].riders

In [ ]:
df_elevator = pd.DataFrame(elevator_log, columns=['floor', 'direction','time'])
df_elevator

In [108]:
df_customer = pd.DataFrame(customers_log, columns=['cid', 'source','destination','start_time', 'boarding_time', 'leave_time'])
df_customer['waiting_time'] = df_customer['boarding_time'] - df_customer['start_time']
df_customer['journey_time'] = df_customer['leave_time'] -df_customer['start_time']

print(df_customer['waiting_time'].mean(),df_customer['journey_time'].mean())
df_customer



131.9527153821419 180.13206670359259


,cid,source,destination,start_time,boarding_time,leave_time,waiting_time,journey_time
0,2,3,1,50.151414,181.564992,202.564992,131.413578,152.413578
1,1,5,1,13.174258,159.564992,202.564992,146.390734,189.390734
2,4,6,1,108.564992,148.564992,202.564992,40.000000,94.000000
3,5,B1,3,173.057358,255.564992,297.456447,82.507634,124.399089
4,3,B1,5,92.254119,255.564992,322.456447,163.310873,230.202327
5,15,3,6,234.838180,298.456447,339.456447,63.618267,104.618267
6,17,2,6,276.456447,286.456447,339.456447,10.000000,63.000000
7,20,5,7,298.173208,323.456447,351.456447,25.283239,53.283239
8,7,5,8,182.598294,323.456447,364.456447,140.858153,181.858153
9,10,4,8,201.416817,310.456447,364.456447,109.039630,163.039630
